In [1]:
import spotipy
import spotipy.util as util
import os
from json import load

# make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

# Retrieving our credentials
Before you can get started you need to head over to <https://developer.spotify.com/dashboard/> and create an app in order to be able to get an authorization token to be able to connect to your account. I put a tutorial on how to do this in the README if you haven't already seen it. If you have already done this step then all you need to do is save the client id, client secret, redirect uri, and your spotify username into a dictionary.

creds = {'client_id': 'YOUR CLIENT ID', 'client_secret': 'YOUR CLIENT SECRET', 
        'redirect_uri': 'http://localhost/', 'username': 'YOUR USERNAME'}
        
I've saved mine into a text file credentials.txt so others can't gain access to my account so for your purposes you can ignore the cell directly below. I;m just loading the dictionary from the pre-saved text file.


In [2]:
with open('credentials.txt') as fd:
    creds = load(fd)

## SCOPE

You can define a scope which tells your application how much visibility/access you want to your account. Meaning different scopes give you different permissions. Some allow you to modify playlists while others only grant you reading abilities. You can read all about scopes [here](https://developer.spotify.com/documentation/general/guides/scopes/)

In [3]:
# For the purposes of this notebook we only want to read from a users library
scope = 'user-library-read'

Now that we have our scope and credentials defined we can finally get our token in order to create our Spotiy object.

In [4]:
def get_token(scope, **kwargs):
    try:
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    except:
        os.remove(f".cache-{kwargs['username']}")
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    return token

In [5]:
token = get_token(scope, **creds)

In [6]:
# Now that we have our token we can create our Spotify object.
sp = spotipy.Spotify(auth=token)

In [7]:
# some of the methods require the user id so its a good idea to save it now 
user = sp.current_user()
user_id = user['id']

Now that we have everything we need we can use sp.search to search for tracks, albums, playlists, or artists by name.

In [8]:
sp.search('Jazz Vibes', type='playlist')

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=Jazz+Vibes&type=playlist&market=US&offset=0&limit=10',
  'items': [{'collaborative': False,
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX0SM0LYsmbMT'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0SM0LYsmbMT',
    'id': '37i9dQZF1DX0SM0LYsmbMT',
    'images': [{'height': None,
      'url': 'https://pl.scdn.co/images/pl/default/d376cd0266b552035f73c458face3ddf88a7761e',
      'width': None}],
    'name': 'Jazz Vibes',
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTU2NDQwMTk0OSwwMDAwMDExYjAwMDAwMTZjM2Q5ZjViNTMwMDAwMDE2ODk5YTUyM2Zj',
    'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX

`sp.search` defaults to returning the first 10 results that match the search query, but if you want a more accurate search you can search music by their id or uri. To do this you can go to the spotify app and find the album, artist, playlist, or track and click on the three dots, hit **Share** and finally **Copy Spotify URI**

![](uri.png)

# Searching for a playlist by its ID

Just follow the steps above to get the URI, which in the case of Jazz Vibes is **spotify:playlist:37i9dQZF1DX0SM0LYsmbMT**

In [9]:
playlist_info = sp.user_playlist(user_id, playlist_id='spotify:playlist:37i9dQZF1DX0SM0LYsmbMT')
print(type(playlist_info))
playlist_info

<class 'dict'>


{'collaborative': False,
 'description': "Chill hip-hop beats with a jazzy edge. Don't call it lo-fi!",
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX0SM0LYsmbMT'},
 'followers': {'href': None, 'total': 1409310},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0SM0LYsmbMT',
 'id': '37i9dQZF1DX0SM0LYsmbMT',
 'images': [{'height': None,
   'url': 'https://pl.scdn.co/images/pl/default/d376cd0266b552035f73c458face3ddf88a7761e',
   'width': None}],
 'name': 'Jazz Vibes',
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MTU2NDQwMTk0OSwwMDAwMDExYjAwMDAwMTZjM2Q5ZjViNTMwMDAwMDE2ODk5YTUyM2Zj',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0SM0LYsmbMT/tracks?offset=0&limit=100',
  'items': [{'a

We see that the results are a dictionary with a lot of information we can narrow down our search by looking at the keys lets try to compile a list of all the songs and their associated ids in the playlist

In [10]:
# tracks seems like the most reasonable place to search for the track names / ids
# the first level 'name' and 'id' keys will return the name and id of the playlist itself
playlist_info.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [11]:
playlist_info['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [12]:
# items is the only key that yields more information the rest just return a single value

# items is a list where each item refers to a track
print(type(playlist_info['tracks']['items']))
print(len(playlist_info['tracks']['items']))
playlist_info['tracks']['items'][0]

<class 'list'>
100


{'added_at': '2019-07-29T12:04:51Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
  'href': 'https://api.spotify.com/v1/users/',
  'id': '',
  'type': 'user',
  'uri': 'spotify:user:'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4smeBjKru2y3oeQkGSfotv'},
     'href': 'https://api.spotify.com/v1/artists/4smeBjKru2y3oeQkGSfotv',
     'id': '4smeBjKru2y3oeQkGSfotv',
     'name': '7apes',
     'type': 'artist',
     'uri': 'spotify:artist:4smeBjKru2y3oeQkGSfotv'}],
   'available_markets': ['AD',
    'AE',
    'AR',
    'AT',
    'AU',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HK',
    'HN',
    'HU',
    'ID',
    'IE',
    'I

In [13]:
# 'track seems like the next most logical place to go'
playlist_info['tracks']['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [14]:
# finally 'name' should give us the name of the first track in the playlist
playlist_info['tracks']['items'][0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [15]:
playlist_info['tracks']['items'][0]['track']['name']

'Glow'

In [16]:
# and 'id' should give us the id of the first track
playlist_info['tracks']['items'][0]['track']['id']

'2O0ZxPTJLVj1R8dZ0IjiM7'

Now that we know how to get the names and ids of every track in a playlist we can define a function where we input the playlist id and get two lists with the first containing the names and the second the ids

In [17]:
def get_playlist_track_names_ids(playlist_id):
    """
    This function returns a list of tuples where the first element of the tuple
    are the names of every track and the second are the associated track ids. 
    """
    global user_id
    playlist_info = sp.user_playlist(user_id, playlist_id=playlist_id)
    tracks = playlist_info['tracks']['items']
    names, ids = [], []
    
    for track in tracks:
        names.append(track['track']['name'])
        ids.append(track['track']['id'])
    
    return list(zip(names, ids))

In [18]:
songs = get_playlist_track_names_ids('spotify:playlist:37i9dQZF1DX0SM0LYsmbMT')
songs[:10]

[('Glow', '2O0ZxPTJLVj1R8dZ0IjiM7'),
 ('Coral Dreams', '2QLoxnf8H9MVHdlJt6l3Pe'),
 ('Never Knew', '3TS4oj5Jfj1zxR4Dn81JHV'),
 ('Blue Voyager', '7BVWBbNukEsUpFeB2OCrhK'),
 ('Clouds on My Bed', '1pab9zVbWB1AsOknBXifbd'),
 ('Caps', '1VvJrVBlqNrBxXdy1akslD'),
 ('Paperplanes', '7Ah5dqNBn8OpGpg2YZGzuX'),
 ('Inglewood Sunset', '4StT5zVHtsvvEwfPupfSIO'),
 ("At Dizzy's", '0UHgJn9wcpxZ8r90Ospbrh'),
 ('Apple Pie', '3Z9rY3eDTK2wRjGcMr3pIJ')]

And if need be we can unzip this into two lists of names and ids

In [19]:
names, ids = zip(*songs)
print(names[:3])
print(ids[:3])

('Glow', 'Coral Dreams', 'Never Knew')
('2O0ZxPTJLVj1R8dZ0IjiM7', '2QLoxnf8H9MVHdlJt6l3Pe', '3TS4oj5Jfj1zxR4Dn81JHV')


#### We now know how to get the songs from a playlist the process to get songs from albums and artists are slightly different but we can use the same workflow to narrow down where to search in the dictionary for the information you want.